In [98]:
"""Necessary Imports and Package for Pylint"""
import tkinter
from tkinter import filedialog
import os
import time
import math
import csv
import pandas
#!pip install nbqa pylint

In [99]:
class Calculator:
    """This is a representation of the Calculator Class"""
    history_calculations = []
    @staticmethod
    def add_number(tuple_of_numbers: tuple):
        """ Instantiating Addition object and passing value a and b to the constructor"""
        sum_of_values = 0.0
        for value in tuple_of_numbers:
            sum_of_values = value + sum_of_values
        Calculator.add_to_history(sum_of_values)
    @staticmethod
    def subtract_number(tuple_of_numbers: tuple):
        """ subtract number from result"""
        sum_of_values = tuple_of_numbers.values[0]
        for value in tuple_of_numbers.values[1:len(tuple_of_numbers.values)]:
            sum_of_values = sum_of_values - value
        Calculator.add_to_history(sum_of_values)
    @staticmethod
    def multiple_number(tuple_of_numbers: tuple):
        """ multiply number from result"""
        sum_of_values = 1.0
        for value in tuple_of_numbers.values:
            sum_of_values = sum_of_values * value
        Calculator.add_to_history(sum_of_values)
    @staticmethod
    def divide_number(tuple_of_numbers: tuple):
        """ divide number from result"""
        sum_of_values = tuple_of_numbers.values[0]
        for value in tuple_of_numbers.values[1:len(tuple_of_numbers.values)]:
            try:
                sum_of_values =  sum_of_values / value
            except ArithmeticError:
                return ArithmeticError('Division by Zero')
        Calculator.add_to_history(sum_of_values)
    @staticmethod
    def add_to_history(result_values):
        """Add result to history"""
        Calculator.history_calculations.append(result_values)
        return True
    @staticmethod
    def get_calculator_result():
        """Returns the last value added to the calculation list"""
        return Calculator.history_calculations[-1]

In [271]:
class Importer: 
    """This the CSV Importer Class"""
    @staticmethod
    def operation_name_to_function(operation_string_name):
        """Contains dictionary to convert string to Calculator method"""
        operation_dictionary = {"addition": Calculator.add_number, "subtraction": Calculator.subtract_number,
                                "multiplication": Calculator.multiple_number,"division": Calculator.divide_number}
        return operation_dictionary.get(operation_string_name)
    @staticmethod
    def calculate_data_file():
        """Prompts user for file selection to processes file(s) """
        return Importer.perform_calculation_on_data_file(Importer.file_dialog_navigator())
    @staticmethod
    def file_dialog_navigator():
        """Opens tkinter File Dialog for User to Select .csv data files"""
        data_file_list = []
        root_path = tkinter.Tk()
        root_path.title('Calculator File system Import')
        root_path.withdraw()
        data_file_list = filedialog.askopenfiles(mode='r', filetypes=[('CSV Files', '*.csv')])
        return data_file_list
    @staticmethod
    def perform_calculation_on_data_file(selected_data_files):
        """ Takes dictionary of operation and dataframe for per row calculation"""
        data_file_batch = []
        error_catch=[]
        data_file_counter = 0
        print("... Preprocessing Datafile Batch! ...")
        for data_operation, data_frame_values, current_file_name in Importer.data_file_zipped(selected_data_files):
            calculator_method_call = Importer.operation_name_to_function(data_operation)
            results_column = data_frame_values.iloc[:, -1]
            data_to_be_calculated = data_frame_values.drop('result', axis=1)
            current_processing_file_name = []
            current_data_operation = []
            current_processing_time = []
            current_calculation_record_number = []
            calculator_row_results = []
            error_message =[]
            error_time=[]
            error_file_name=[]
            error_row_index=[]
            for row_index_value, row_data in data_to_be_calculated.iterrows():
                calculator_method_call(row_data[row_data.notna()])
                if math.isinf(Calculator.get_calculator_result()) is False:
                    calculator_row_results.append(Calculator.get_calculator_result()) 
                    current_processing_time.append(Importer.get_unix_time())
                    current_processing_file_name.append(current_file_name)
                    current_calculation_record_number.append(row_index_value)
                    current_data_operation.append(data_operation)
                else:
                    error_message.append("DIV_ZERO")
                    error_time.append(Importer.get_unix_time())
                    error_file_name.append(current_file_name)
                    error_row_index.append(row_index_value)   
            if len(error_time) != 0:  
                error_catch.append([zip(error_message,error_time,error_file_name,error_row_index),selected_data_files,data_file_counter])
            data_file_batch.append([zip(current_calculation_record_number, 
                                        current_processing_file_name, current_processing_time,
                                        current_data_operation, calculator_row_results,results_column),
                                        selected_data_files,data_file_counter])
            data_file_counter +=1
            print("Currently processing: "+current_file_name+". File: "+
                  str(data_file_counter)+ " of " + str(len(selected_data_files))) 
        print( "\n... Commencing Validation Process ...\n")
        Importer.write_error_log(error_catch)
        Importer.write_result_to_csv(data_file_batch)
        print( "\n... Preparing to Write Results ...\n")
        print("Processing Complete!")
        return True
    @staticmethod   
    def write_error_log(file_batch):
        """Logs file with Math Error"""
        for proccesed_item,data_file_list,processing_index in file_batch:
            current_data_file = data_file_list[processing_index]
            results_data_frame = pandas.DataFrame(list(proccesed_item), columns = ['error_message','error_timestamp',
                                                                                   'error_file_name', 'error_calculation_index'])
            csv_file_path = Importer.data_file_to_path(current_data_file).replace('results','error')
            Importer.create_error_directory(csv_file_path)
            result_file_name = 'error_'+current_data_file.name.split("/")[-1]
            processed_file_name_path = os.path.join(csv_file_path,result_file_name)
            try:
                results_data_frame.to_csv(processed_file_name_path)
            except:
                print("Something went wrong")
    @staticmethod   
    def write_result_to_csv(file_batch):
        """Writes calculator result to result folder with unix time stamp, filename, record number, operation"""
        for proccesed_item,data_file_list,processing_index in file_batch:
            current_data_file = data_file_list[processing_index]
            results_data_frame = pandas.DataFrame(list(proccesed_item), columns = ['calculation_record','file_name',
                                                                                   'timestamp', 'operation',
                                                                                   'calculated_results','expected_results'])
            csv_file_path = Importer.data_file_to_path(current_data_file)
            Importer.create_results_directory(csv_file_path)
            result_file_name = 'results_'+current_data_file.name.split("/")[-1]
            Importer.check_sum_data(results_data_frame['calculated_results'],
                           results_data_frame['expected_results'],result_file_name)
            processed_file_name_path = os.path.join(csv_file_path,result_file_name)
            try:
                results_data_frame.to_csv(processed_file_name_path)
            except:
                print("Something went wrong")
    @staticmethod
    def check_sum_data(calculated_results,true_results,file_name):
        """Internall Validates Calculated Result with Result Column"""
        validate_data = pandas.DataFrame(abs(calculated_results)-abs(true_results))
        if (sum(validate_data)) == 0 :
            print("Internal Verification for "+file_name+" Successful!")
        else:
            print("Error: Mismatch in Calculated and Expected Results for "+file_name + " at line(s): "
                  +str(validate_data[validate_data.iloc[0]>0]))
    @staticmethod
    def data_file_zipped(imported_data_files):
        """Processes file operations with data frame"""
        data_file_names = Importer.data_file_path_to_name(imported_data_files)
        data_file_operations_list = Importer.file_name_to_operation(Importer.data_file_path_to_name(imported_data_files))
        data_file_data_frames_list = Importer.data_file_to_dataframe(imported_data_files)
        return zip(data_file_operations_list, data_file_data_frames_list, data_file_names)
    @staticmethod
    def file_name_to_operation(list_of_files):
        """Parses file.name to embedded operation"""
        data_file_operation_list = []
        for file_name in list_of_files:
            data_file_operation_list.append(file_name.split("_")[0])
        return data_file_operation_list
    @staticmethod
    def data_file_to_path(data_file):
        """Takes data file object and parses path"""
        data_file_name_path = data_file.name
        file_name = data_file_name_path.split("/")[-1]
        return data_file_name_path.replace(file_name,'results')  
    @staticmethod
    def create_results_directory(directory_path):
        """Utilizes data_file_path to generate results folder"""
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
        return True
    @staticmethod
    def create_error_directory(directory_path):
        """Utilizes data_file_path to generate results folder"""
        log_path = directory_path.replace('results','error')
        if not os.path.exists(log_path):
            os.makedirs(log_path)
        return True
    @staticmethod
    def data_file_to_dataframe(data_files):
        """Takes list of data files and reads them into a list of data frames"""
        parsed_data_frames = []
        for data in data_files:
            parsed_data_frames.append(pandas.read_csv(data.name))
        return parsed_data_frames
    @staticmethod
    def data_file_path_to_name (data_files_list):
        """Takes list of file paths and returns filename(s) as list"""
        data_file_name_list = []
        for file in data_files_list:
            data_file_name_list.append(file.name.split("/")[-1])
        return data_file_name_list
    @staticmethod
    def get_unix_time():
        """Returns Unix Timestamp"""
        return int(time.time())

In [272]:
def test_csv_importer():
    """Tests the CSV Module"""
    assert Importer.calculate_data_file() is True

In [273]:
test_csv_importer()

... Preprocessing Datafile Batch! ...
Currently processing: addition_large.csv. File: 1 of 13
Currently processing: addition_medium.csv. File: 2 of 13
Currently processing: addition_small.csv. File: 3 of 13
Currently processing: division_error.csv. File: 4 of 13


<ipython-input-99-2974e2869c58>:31: RuntimeWarning: divide by zero encountered in double_scalars
  sum_of_values =  sum_of_values / value


Currently processing: division_large.csv. File: 5 of 13
Currently processing: division_medium.csv. File: 6 of 13
Currently processing: division_small.csv. File: 7 of 13
Currently processing: multiplication_large.csv. File: 8 of 13
Currently processing: multiplication_medium.csv. File: 9 of 13
Currently processing: multiplication_small.csv. File: 10 of 13
Currently processing: subtraction_large.csv. File: 11 of 13
Currently processing: subtraction_medium.csv. File: 12 of 13
Currently processing: subtraction_small.csv. File: 13 of 13

... Commencing Validation Process ...

Internal Verification for results_addition_large.csv Successful!
Internal Verification for results_addition_medium.csv Successful!
Internal Verification for results_addition_small.csv Successful!
Internal Verification for results_division_error.csv Successful!
Internal Verification for results_division_large.csv Successful!
Internal Verification for results_division_medium.csv Successful!
Internal Verification for resu